Loading Libraries

In [1]:
import numba
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import gc  # garbage collector

from fastcore.basics import *
from fastcore.parallel import *
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8

from sklearn.model_selection import train_test_split
from sklearn import metrics  # for accuracy calculation
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from functools import partial
from os import cpu_count
import matplotlib.pyplot as plt

import math
from pylab import imshow, show
from timeit import default_timer as timer

from dask import dataframe as dd
from dask.distributed import Client

Setting up paths to csv files / datasets

In [2]:
# CSV-01-12
path_DdoS_DNS = "C:\\CIC-DDOS-2019\\DrDoS_DNS.csv"
path_DdoS_MSSQL = "C:\\CIC-DDOS-2019\\DrDoS_MSSQL.csv"
path_DdoS_LDAP = "C:\\CIC-DDOS-2019\\DrDoS_LDAP.csv"
path_DdoS_NTP = "C:\\CIC-DDOS-2019\\DrDoS_NTP.csv"
path_DdoS_NetBIOS = "C:\\CIC-DDOS-2019\\DrDoS_NetBIOS.csv"
path_DdoS_SNMP = "C:\\CIC-DDOS-2019\\DrDoS_SNMP.csv"
path_DdoS_SSDP = "C:\\CIC-DDOS-2019\\DrDoS_SSDP.csv"
path_DdoS_UDP = "C:\\CIC-DDOS-2019\\DrDoS_UDP.csv"
path_Syn = "C:\\CIC-DDOS-2019\\Syn.csv"
path_TFTP = "C:\\CIC-DDOS-2019\\TFTP.csv"
path_UDPLag = "C:\\CIC-DDOS-2019\\UDPLag.csv"

# # CSV-03-11
# path__LDAP = "../CICDDoS-2019/CSV-03-11/03-11/LDAP.csv"
# path__MSSQL = "../CICDDoS-2019/CSV-03-11/03-11/MSSQL.csv"
# path__NetBIOS = "../CICDDoS-2019/CSV-03-11/03-11/NetBIOS.csv"
# path__Portmap = "../CICDDoS-2019/CSV-03-11/03-11/Portmap.csv"
# path__Syn = "../CICDDoS-2019/CSV-03-11/03-11/Syn.csv"
# path__UDP = "../CICDDoS-2019/CSV-03-11/03-11/UDP.csv"
# path__UDPLag = "../CICDDoS-2019/CSV-03-11/03-11/UDPLag.csv"

paths = [path_DdoS_DNS, path_DdoS_MSSQL, path_DdoS_LDAP, path_DdoS_NTP, path_DdoS_NetBIOS, path_DdoS_SNMP,
         path_DdoS_SSDP, path_DdoS_UDP, path_Syn, path_TFTP, path_UDPLag]
# , path__LDAP, path__MSSQL, path__NetBIOS,
#      path__Portmap, path__Syn, path__UDP, path__UDPLag]

Column / feature names

In [3]:
col_name_consistency = {
    'Flow ID': 'Flow ID',
    'Source IP': 'Source IP',
    'Src IP': 'Source IP',
    'Source Port': 'Source Port',
    'Src Port': 'Source Port',
    'Destination IP': 'Destination IP',
    'Dst IP': 'Destination IP',
    'Destination Port': 'Destination Port',
    'Dst Port': 'Destination Port',
    'Protocol': 'Protocol',
    'Timestamp': 'Timestamp',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Tot Fwd Pkts': 'Total Fwd Packets',
    'Total Backward Packets': 'Total Backward Packets',
    'Tot Bwd Pkts': 'Total Backward Packets',
    'Total Length of Fwd Packets': 'Fwd Packets Length Total',
    'TotLen Fwd Pkts': 'Fwd Packets Length Total',
    'Total Length of Bwd Packets': 'Bwd Packets Length Total',
    'TotLen Bwd Pkts': 'Bwd Packets Length Total',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Pkt Len Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Pkt Len Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Fwd Pkt Len Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Pkt Len Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Pkt Len Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Pkt Len Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Bwd Pkt Len Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Byts/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow Pkts/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Tot': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Tot': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length',
    'Fwd Header Len': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Bwd Header Len': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Fwd Pkts/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Bwd Pkts/s': 'Bwd Packets/s',
    'Min Packet Length': 'Packet Length Min',
    'Pkt Len Min': 'Packet Length Min',
    'Max Packet Length': 'Packet Length Max',
    'Pkt Len Max': 'Packet Length Max',
    'Packet Length Mean': 'Packet Length Mean',
    'Pkt Len Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Pkt Len Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'Pkt Len Var': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'FIN Flag Cnt': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count',
    'SYN Flag Cnt': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count',
    'RST Flag Cnt': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'PSH Flag Cnt': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'ACK Flag Cnt': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count',
    'URG Flag Cnt': 'URG Flag Count',
    'CWE Flag Count': 'CWE Flag Count',
    'CWE Flag Cnt': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Count',
    'ECE Flag Cnt': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Average Packet Size': 'Avg Packet Size',
    'Pkt Size Avg': 'Avg Packet Size',
    'Avg Fwd Segment Size': 'Avg Fwd Segment Size',
    'Fwd Seg Size Avg': 'Avg Fwd Segment Size',
    'Avg Bwd Segment Size': 'Avg Bwd Segment Size',
    'Bwd Seg Size Avg': 'Avg Bwd Segment Size',
    'Fwd Avg Bytes/Bulk': 'Fwd Avg Bytes/Bulk',
    'Fwd Byts/b Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Avg Packets/Bulk': 'Fwd Avg Packets/Bulk',
    'Fwd Pkts/b Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Avg Bulk Rate': 'Fwd Avg Bulk Rate',
    'Fwd Blk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk': 'Bwd Avg Bytes/Bulk',
    'Bwd Byts/b Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Avg Packets/Bulk': 'Bwd Avg Packets/Bulk',
    'Bwd Pkts/b Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Avg Bulk Rate': 'Bwd Avg Bulk Rate',
    'Bwd Blk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets',
    'Subflow Fwd Pkts': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Subflow Fwd Byts': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets',
    'Subflow Bwd Pkts': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
    'Subflow Bwd Byts': 'Subflow Bwd Bytes',
    'Init_Win_bytes_forward': 'Init Fwd Win Bytes',
    'Init Fwd Win Byts': 'Init Fwd Win Bytes',
    'Init_Win_bytes_backward': 'Init Bwd Win Bytes',
    'Init Bwd Win Byts': 'Init Bwd Win Bytes',
    'act_data_pkt_fwd': 'Fwd Act Data Packets',
    'Fwd Act Data Pkts': 'Fwd Act Data Packets',
    'min_seg_size_forward': 'Fwd Seg Size Min',
    'Fwd Seg Size Min': 'Fwd Seg Size Min',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
    'Label': 'Label'
}

Following Columns may have little insignificance over model

In [4]:
drop_columns = [  # this list includes all spellings across CIC NIDS datasets
    "Flow ID",
    'Fwd Header Length.1',
    "Source IP",
    "Src IP",
    "Source Port",
    "Src Port",
    "Destination IP",
    "Dst IP",
    "Destination Port",
    "Dst Port",
    "Timestamp",
    "Unnamed: 0",
    "Inbound",
    "SimillarHTTP"  # CIC-DDoS other undocumented columns
]
len(drop_columns)  # src_port,dst_port, src_ip, dst_ip these are duplicate

14

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
tf.config.experimental.list_physical_devices('GPU')
# import cudf
# dtypes = {'SimillarHTTP': 'object'}
# df_gpu = cudf.read_csv(path_TFTP, blocksize=50e6, low_memory=False, dtype=dtypes)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13763483456108361193
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2966106932
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5093324869116794965
physical_device_desc: "device: 0, name: NVIDIA GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
def readHugeCsvFileAsDataFrame(file_path):
    # start = timer()
    dtypes = {'SimillarHTTP': 'object',
              'Timestamp': 'object',
              'Source IP': 'str',
              'Destination IP': 'str',
              'Flow ID': 'object',
              'Label': 'object',
              }
    for feature in [f'f_{i}' for i in range(82)]:
        dtypes[feature] = "float32"

    dask_df = dd.read_csv(file_path, low_memory=False, dtype=dtypes)  # 50MB chunk-size
    # elapsed_time = timer() - start
    # print("Read csv with dask: ", elapsed_time, "sec")
    return dask_df.compute()

In [7]:
scheduler = Client()
scheduler

Client Scheduler: tcp://127.0.0.1:12097 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.47 GB


Without Scheduler (see the elapsed time to process)

In [8]:
%%time
# workingDataFrame = readHugeCsvFileAsDataFrame(path_UDPLag)
workingDataFrame = readHugeCsvFileAsDataFrame(path_Syn)
# workingDataFrame.describe()
workingDataFrame

Wall time: 1min 20s


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,281052,172.16.0.5-192.168.50.1-53058-53058-6,172.16.0.5,53058,192.168.50.1,53058,6,2018-12-01 13:30:30.741451,115799309,19,...,646237.483665,1709809.0,1.0,1.426117e+07,3.220326e+06,21714933.0,11043464.0,0,1,Syn
1,450424,172.16.0.5-192.168.50.1-32237-32237-6,172.16.0.5,32237,192.168.50.1,32237,6,2018-12-01 13:30:30.741452,113973933,16,...,19.595918,49.0,1.0,1.628198e+07,2.573891e+06,20019405.0,11993631.0,0,1,Syn
2,182979,172.16.0.5-192.168.50.1-60495-9840-6,172.16.0.5,60495,192.168.50.1,9840,6,2018-12-01 13:30:30.741501,112,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
3,41540,172.16.0.5-192.168.50.1-59724-59724-6,172.16.0.5,59724,192.168.50.1,59724,6,2018-12-01 13:30:30.741563,105985004,16,...,17.705259,48.0,1.0,1.514071e+07,3.077366e+06,20954123.0,11120336.0,0,1,Syn
4,358711,172.16.0.5-192.168.50.1-60496-32538-6,172.16.0.5,60496,192.168.50.1,32538,6,2018-12-01 13:30:30.741565,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157751,432793,172.16.0.5-192.168.50.1-43299-42563-6,172.16.0.5,43299,192.168.50.1,42563,6,2018-12-01 13:34:27.402384,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
157752,514566,172.16.0.5-192.168.50.1-43300-58948-6,172.16.0.5,43300,192.168.50.1,58948,6,2018-12-01 13:34:27.402386,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
157753,147509,172.16.0.5-192.168.50.1-43301-52300-6,172.16.0.5,43301,192.168.50.1,52300,6,2018-12-01 13:34:27.402451,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
157754,46401,172.16.0.5-192.168.50.1-43302-40846-6,172.16.0.5,43302,192.168.50.1,40846,6,2018-12-01 13:34:27.403142,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn


With Scheduler (see the elapsed time to process)

%%time
workingDataFrame = scheduler.submit(readHugeCsvFileAsDataFrame, path_UDPLag)
df = workingDataFrame.result()
df

In [9]:
workingDataFrame.columns

Index(['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Len

In [10]:
workingDataFrame.isnull().sum()

Unnamed: 0         0
Flow ID            0
 Source IP         0
 Source Port       0
 Destination IP    0
                  ..
 Idle Max          0
 Idle Min          0
SimillarHTTP       0
 Inbound           0
 Label             0
Length: 88, dtype: int64

Dropping Unnecessary Features

In [11]:
workingDataFrame.columns = workingDataFrame.columns.str.strip()  # sometimes there's leading / trailing whitespace
workingDataFrame.drop(columns=drop_columns, inplace=True, errors='ignore')
workingDataFrame.rename(columns=col_name_consistency, inplace=True)

workingDataFrame.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,115799309,19,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,244280.142857,646237.483665,1709809.0,1.0,1.426117e+07,3.220326e+06,21714933.0,11043464.0,Syn
1,6,113973933,16,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,9.000000,19.595918,49.0,1.0,1.628198e+07,2.573891e+06,20019405.0,11993631.0,Syn
2,6,112,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Syn
3,6,105985004,16,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,7.857143,17.705259,48.0,1.0,1.514071e+07,3.077366e+06,20954123.0,11120336.0,Syn
4,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Syn


In [12]:
# workingDataFrame.dtypes
workingDataFrame.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1582681 entries, 0 to 157755
Data columns (total 78 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Protocol                  1582681 non-null  int64  
 1   Flow Duration             1582681 non-null  int64  
 2   Total Fwd Packets         1582681 non-null  int64  
 3   Total Backward Packets    1582681 non-null  int64  
 4   Fwd Packets Length Total  1582681 non-null  float64
 5   Bwd Packets Length Total  1582681 non-null  float64
 6   Fwd Packet Length Max     1582681 non-null  float64
 7   Fwd Packet Length Min     1582681 non-null  float64
 8   Fwd Packet Length Mean    1582681 non-null  float64
 9   Fwd Packet Length Std     1582681 non-null  float64
 10  Bwd Packet Length Max     1582681 non-null  float64
 11  Bwd Packet Length Min     1582681 non-null  float64
 12  Bwd Packet Length Mean    1582681 non-null  float64
 13  Bwd Packet Length Std     15

Data Cleaning Based on Data Types (DownSizing)

In [13]:
for column in workingDataFrame:
    if workingDataFrame[column].dtype == 'float64':
        workingDataFrame[column] = pd.to_numeric(workingDataFrame[column], downcast='float')
    if workingDataFrame[column].dtype == 'int64':
        workingDataFrame[column] = pd.to_numeric(workingDataFrame[column], downcast='integer')

workingDataFrame.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1582681 entries, 0 to 157755
Data columns (total 78 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Protocol                  1582681 non-null  int8   
 1   Flow Duration             1582681 non-null  int32  
 2   Total Fwd Packets         1582681 non-null  int16  
 3   Total Backward Packets    1582681 non-null  int16  
 4   Fwd Packets Length Total  1582681 non-null  float32
 5   Bwd Packets Length Total  1582681 non-null  float32
 6   Fwd Packet Length Max     1582681 non-null  float32
 7   Fwd Packet Length Min     1582681 non-null  float32
 8   Fwd Packet Length Mean    1582681 non-null  float32
 9   Fwd Packet Length Std     1582681 non-null  float32
 10  Bwd Packet Length Max     1582681 non-null  float32
 11  Bwd Packet Length Min     1582681 non-null  float32
 12  Bwd Packet Length Mean    1582681 non-null  float32
 13  Bwd Packet Length Std     15

Removing NaN values

In [14]:
workingDataFrame.isna().any(axis=1).sum()

202277

In [15]:
workingDataFrame.replace([np.inf, -np.inf], np.nan, inplace=True)
workingDataFrame.dropna(inplace=True)

After Dropping NaN values, want to see total rows.
That's why, Converting pandas dataFrame into dask's dataFrame

In [16]:
ddf = dd.from_pandas(workingDataFrame, npartitions=10)
ddf.compute()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,115799309,19,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,244280.140625,646237.5,1709809.0,1.0,14261168.0,3220326.25,21714932.0,11043464.0,Syn
0,6,48,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn
0,6,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn
0,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn
0,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164691,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn
164692,6,97,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn
164693,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn
164694,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Syn


Dropping Duplicates
There should be no duplicates because they can bias training and can lead to over-optimistic estimates of classification performance during testing.

In [17]:
workingDataFrame.duplicated().sum()

1224497

Fully duplicate rows to be removed

In [18]:
workingDataFrame.drop_duplicates(inplace=True)
workingDataFrame.reset_index(inplace=True, drop=True)
workingDataFrame

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,115799309,19,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,244280.140625,646237.500000,1709809.0,1.0,14261168.0,3220326.25,21714932.0,11043464.0,Syn
1,6,113973933,16,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,9.000000,19.595919,49.0,1.0,16281983.0,2573891.00,20019404.0,11993631.0,Syn
2,6,112,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
3,6,105985004,16,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,7.857143,17.705259,48.0,1.0,15140707.0,3077366.50,20954124.0,11120336.0,Syn
4,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155862,6,111,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155863,6,162,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155864,6,218,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155865,6,105,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn


After data cleaning, how much storage it holds now

In [19]:
workingDataFrame.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155867 entries, 0 to 155866
Data columns (total 78 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Protocol                  155867 non-null  int8   
 1   Flow Duration             155867 non-null  int32  
 2   Total Fwd Packets         155867 non-null  int16  
 3   Total Backward Packets    155867 non-null  int16  
 4   Fwd Packets Length Total  155867 non-null  float32
 5   Bwd Packets Length Total  155867 non-null  float32
 6   Fwd Packet Length Max     155867 non-null  float32
 7   Fwd Packet Length Min     155867 non-null  float32
 8   Fwd Packet Length Mean    155867 non-null  float32
 9   Fwd Packet Length Std     155867 non-null  float32
 10  Bwd Packet Length Max     155867 non-null  float32
 11  Bwd Packet Length Min     155867 non-null  float32
 12  Bwd Packet Length Mean    155867 non-null  float32
 13  Bwd Packet Length Std     155867 non-null  f

In [20]:
columnList = workingDataFrame.columns.tolist()
# columnList[:-1]  # just for now, Omitting 'label' column

from sklearn.feature_selection import VarianceThreshold

var_thr = VarianceThreshold(threshold=0.1)  #Removing both constant and quasi-constant
var_thr.fit(workingDataFrame[columnList[:-1]])

var_threshold_bool_list = var_thr.get_support()
var_threshold_bool_list_after_label_added = np.append(var_threshold_bool_list,True)
var_threshold_bool_list_after_label_added

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [21]:
concol = [column for column in workingDataFrame.columns
          if column not in workingDataFrame.columns[var_threshold_bool_list_after_label_added]]

for omittedFeatures in concol:
    print(omittedFeatures)

workingDataFrame.drop(columns=concol, inplace=True, errors='ignore')
workingDataFrame

Fwd PSH Flags
Bwd PSH Flags
Fwd URG Flags
Bwd URG Flags
FIN Flag Count
SYN Flag Count
RST Flag Count
PSH Flag Count
ACK Flag Count
URG Flag Count
CWE Flag Count
ECE Flag Count
Down/Up Ratio
Fwd Avg Bytes/Bulk
Fwd Avg Packets/Bulk
Fwd Avg Bulk Rate
Bwd Avg Bytes/Bulk
Bwd Avg Packets/Bulk
Bwd Avg Bulk Rate


,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,115799309,19,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,244280.140625,646237.500000,1709809.0,1.0,14261168.0,3220326.25,21714932.0,11043464.0,Syn
1,6,113973933,16,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,9.000000,19.595919,49.0,1.0,16281983.0,2573891.00,20019404.0,11993631.0,Syn
2,6,112,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
3,6,105985004,16,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,7.857143,17.705259,48.0,1.0,15140707.0,3077366.50,20954124.0,11120336.0,Syn
4,6,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155862,6,111,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155863,6,162,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155864,6,218,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155865,6,105,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn


Now arising problem is that, row number has been significantly reduced but feature numbers are still 78!!
So, need feature engineering here

In [22]:

def featueEngineeringBasedOnZero(dataFrameArg, thresholdPercentage, showPercentage):
    totalCols = dataFrameArg.shape[1]
    totalRows = len(dataFrameArg)
    unNecessaryFeatureCount = 0
    unNecessaryFeatureNames = []

    for column in dataFrameArg:
        zerosInCol = (dataFrameArg[column] == 0).sum()
        if zerosInCol != 0:
            percentageOfZerosInRow = ((zerosInCol * 100) / totalRows)

            if showPercentage:
                print(column, "\t\t-\t\t", zerosInCol, "\t\t-\t\t", percentageOfZerosInRow)

            if percentageOfZerosInRow > thresholdPercentage:
                unNecessaryFeatureNames.append(column)
                unNecessaryFeatureCount = unNecessaryFeatureCount + 1

    print("\nTotal features having more than ", thresholdPercentage, "% zero are - ", unNecessaryFeatureCount,
          "out of ",
          totalCols)
    return unNecessaryFeatureNames

Identifying those features containing 99% zeroes

In [23]:
featureContainingAlmostZero = featueEngineeringBasedOnZero(dataFrameArg=workingDataFrame, thresholdPercentage=99,
                                                           showPercentage=False)
featureContainingAlmostZero


Total features having more than  99 % zero are -  22 out of  59


['Fwd Packets Length Total',
 'Bwd Packets Length Total',
 'Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Min',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Flow Bytes/s',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Variance',
 'Avg Packet Size',
 'Avg Fwd Segment Size',
 'Avg Bwd Segment Size',
 'Subflow Fwd Bytes',
 'Subflow Bwd Bytes',
 'Fwd Act Data Packets']

Omitting above features containing 99% zeroes

In [24]:
workingDataFrame.drop(columns=featureContainingAlmostZero, inplace=True, errors='ignore')
workingDataFrame.rename(columns=col_name_consistency, inplace=True)
workingDataFrame.reset_index(inplace=True, drop=True)
workingDataFrame

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,115799309,19,2,1.813482e-01,5.789966e+06,7.370248e+06,21714932.0,0.0,115799312.0,...,20,244280.140625,646237.500000,1709809.0,1.0,14261168.0,3220326.25,21714932.0,11043464.0,Syn
1,6,113973933,16,0,1.403830e-01,7.598262e+06,8.575157e+06,20019404.0,0.0,113973936.0,...,20,9.000000,19.595919,49.0,1.0,16281983.0,2573891.00,20019404.0,11993631.0,Syn
2,6,112,2,2,3.571429e+04,3.733333e+01,6.379916e+01,111.0,0.0,1.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
3,6,105985004,16,0,1.509648e-01,7.065667e+06,8.074003e+06,20954124.0,1.0,105985008.0,...,20,7.857143,17.705259,48.0,1.0,15140707.0,3077366.50,20954124.0,11120336.0,Syn
4,6,1,2,0,2.000000e+06,1.000000e+00,0.000000e+00,1.0,1.0,1.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155862,6,111,2,2,3.603604e+04,3.700000e+01,3.404409e+01,67.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155863,6,162,2,2,2.469136e+04,5.400000e+01,5.524491e+01,112.0,2.0,48.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155864,6,218,2,2,1.834862e+04,7.266666e+01,1.249973e+02,217.0,0.0,1.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155865,6,105,2,2,3.809524e+04,3.500000e+01,3.031501e+01,53.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn


Now, To see the number of unique values in each column

In [25]:
workingDataFrame.nunique(axis=0)

Protocol                       3
Flow Duration             149455
Total Fwd Packets             45
Total Backward Packets        50
Flow Packets/s            148600
Flow IAT Mean             148983
Flow IAT Std              150557
Flow IAT Max              148429
Flow IAT Min                 200
Fwd IAT Total             147058
Fwd IAT Mean              148303
Fwd IAT Std               148606
Fwd IAT Max               148197
Fwd IAT Min                  181
Bwd IAT Total              20460
Bwd IAT Mean               20478
Bwd IAT Std                20399
Bwd IAT Max                20459
Bwd IAT Min                  104
Fwd Header Length             71
Bwd Header Length             65
Fwd Packets/s             148159
Bwd Packets/s              66231
Subflow Fwd Packets           45
Subflow Bwd Packets           50
Init Fwd Win Bytes            64
Init Bwd Win Bytes            69
Fwd Seg Size Min               6
Active Mean                 8946
Active Std                 25469
Active Max

In [26]:
workingDataFrame

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,115799309,19,2,1.813482e-01,5.789966e+06,7.370248e+06,21714932.0,0.0,115799312.0,...,20,244280.140625,646237.500000,1709809.0,1.0,14261168.0,3220326.25,21714932.0,11043464.0,Syn
1,6,113973933,16,0,1.403830e-01,7.598262e+06,8.575157e+06,20019404.0,0.0,113973936.0,...,20,9.000000,19.595919,49.0,1.0,16281983.0,2573891.00,20019404.0,11993631.0,Syn
2,6,112,2,2,3.571429e+04,3.733333e+01,6.379916e+01,111.0,0.0,1.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
3,6,105985004,16,0,1.509648e-01,7.065667e+06,8.074003e+06,20954124.0,1.0,105985008.0,...,20,7.857143,17.705259,48.0,1.0,15140707.0,3077366.50,20954124.0,11120336.0,Syn
4,6,1,2,0,2.000000e+06,1.000000e+00,0.000000e+00,1.0,1.0,1.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155862,6,111,2,2,3.603604e+04,3.700000e+01,3.404409e+01,67.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155863,6,162,2,2,2.469136e+04,5.400000e+01,5.524491e+01,112.0,2.0,48.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155864,6,218,2,2,1.834862e+04,7.266666e+01,1.249973e+02,217.0,0.0,1.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn
155865,6,105,2,2,3.809524e+04,3.500000e+01,3.031501e+01,53.0,0.0,0.0,...,20,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,Syn


Saving New DataFrame as csv file to new location

In [27]:
def dataCleaningResultToAnotherCSV(dataFrameArg, dirPath, file_name):
    dataFrameArg.to_csv(dirPath + file_name)

In [28]:
newCsvPath = "C:\\CIC-DDOS-2019\\AfterDataCleaning\\"
# newFileName = "UDPLag.csv"
newFileName = "Syn.csv"
dataCleaningResultToAnotherCSV(dataFrameArg=workingDataFrame, dirPath=newCsvPath, file_name=newFileName)

In [29]:
gc.collect()

147